In [256]:
pip install textfsm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     -------------------------------------- 44.7/44.7 kB 200.1 kB/s eta 0:00:00
     ------------------------------------ 840.9/840.9 kB 209.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492055 sha256=463ec9aeaa9b4fead206f53f0423663c5db85d87acb7574899f5573fa64bdbd0
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\6e\54\16\2ab500319d13efeeee4b2007aab5a01589dc775fb1a1f617c5
Successfully built future
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
 
import telnetlib
from textfsm import TextFSM
import re,json


In [39]:
def readCfg(filename):
    """读取配置文件"""
    # 读取json文件内容,返回字典格式
    json_data = {}
    with open(filename,'r',encoding='utf8')as fp:
        json_data = json.load(fp)
    return json_data
cfg = readCfg('sw_config.json')
cfg

{'交换机列表': [{'type': 'hws3700',
   'ip': '192.168.11.1',
   'user': 'admin',
   'passwd': 'wsf2010'},
  {'type': 'hws3700',
   'ip': '192.168.11.200',
   'user': '',
   'passwd': 'wsf2010'}],
 '输出路径': './'}

In [20]:

host='192.168.11.1'

user='admin'
password1='wsf2010'

 
tn=telnetlib.Telnet(host)
tn.read_until(b"Username:")





b'\r\r\n\r\nLogin authentication\r\n\r\n\r\nUsername:'

In [21]:
 

def hwlogin(user,passwd,ip):
    """登入交换机 返回telnet"""
    tn=telnetlib.Telnet(host)
    out = tn.read_until(b"Username:",timeout=5)
    tn.write(user.encode('ascii')+b"\n")
    out = tn.read_until(b"Password:",timeout=5)
    tn.write(passwd.encode('ascii')+b"\n")
    outLs = tn.expect([b"\r\n<.+?>"],timeout=5)##等待直到读到预期的值
    print(outLs[2])
    return tn

    
    

In [22]:
tn = hwlogin(user,password1,host)

b'\r\nInfo: The max number of VTY users is 5, and the number\r\n      of current VTY users on line is 3.\r\n<3700>'


In [23]:
 
def readMore():
    """ 读取更多 """
    pass
def list2str(tabLs):
    tabStr = ''.join(tabLs)
    tabStr =  re.sub("\s+---- More ----\x1b\[42D\s+\x1b\[42D", "\r\n", tabStr)
    tabStr =  re.sub("\r\n\s{30,}","  ", tabStr)   
    return tabStr
def hwArpTab(tn):
    """ 返回交换 arp表"""
    tabLs = []
    tn.write(b"display arp\n")
    while 1 :
        outLs = tn.expect([b"\r\n<.+?>",b"---- More ----"],timeout=5) 
        if outLs[0] == 1:
            #还有下一个页
            tn.write(b"   ")
            tabLs.append(outLs[2].decode('ascii'))
            #print(outLs[2])
        if outLs[0] <= 0:
            # 完成读取
            #print(outLs[2])
            tabLs.append(outLs[2].decode('ascii'))
            break
 
    return tabLs
tabLs = hwArpTab(tn)
tabStr = list2str(tabLs)                          


In [24]:
print(tabStr)

display arp
IP ADDRESS      MAC ADDRESS  EXPIRE(M) TYPE INTERFACE      VPN-INSTANCE         VLAN 
------------------------------------------------------------------------------
192.168.11.1    80fb-06c9-6db9         I -  Vlanif11
192.168.11.199  b42e-9968-4090  9      D-0  Eth0/0/1  11
192.168.11.30   4419-b641-7795  9      D-0  Eth0/0/1  11
192.168.11.189  0800-2740-3822  9      D-0  Eth0/0/1  11
192.168.12.1    80fb-06c9-6db9         I -  Vlanif12
192.168.13.1    80fb-06c9-6db9         I -  Vlanif13
192.168.14.1    80fb-06c9-6db9         I -  Vlanif14
192.168.14.13   8269-c0e2-cbe1  4      D-0  Eth0/0/11  14
192.168.15.1    80fb-06c9-6db9         I -  Vlanif15
192.168.15.254  8ca6-df43-6b43  9      D-0  Eth0/0/5  15
192.168.15.33   7427-eae8-fef9  9      D-0  Eth0/0/5  15
192.168.15.177  4ced-fb6c-ef9f  9      D-0  Eth0/0/5  15
192.168.15.200  408d-5c8f-2aca  9      D-0  Eth0/0/5  15
192.168.15.88   b42e-99e6-3c49  17     D-0  Eth0/0/5  15
192.168.15.63   4ced-fb41-fba5  17     D-0  

In [25]:


output = '''
                  display arp
IP ADDRESS      MAC ADDRESS  EXPIRE(M) TYPE INTERFACE      VPN-INSTANCE         VLAN 
------------------------------------------------------------------------------
192.168.11.1    80fb-06c9-6db9         I -  Vlanif11
192.168.11.199  b42e-9968-4090  11     D-0  Eth0/0/1  11
192.168.88.253  4cbd-8f90-7198  1      D-0  Eth0/0/11  88
192.168.88.161  Incomplete      0      D-0  Eth0/0/1  88
192.168.88.253  4cbd-8f90-7198  1      D-0  Eth0/0/11  88
192.168.88.161  Incomplete      0      D-0  Eth0/0/1  88
192.168.88.52   Incomplete      0      D-0  Eth0/0/1  88
192.168.0.1     80fb-06c9-6db9         I -  Vlanif100
192.168.0.45    2cea-7feb-b118  20     D-0  Eth0/0/20  100
192.168.0.252   f4ee-0823-956a  20     D-0  Eth0/0/24  100
192.168.0.177   9ca6-15d8-f7a2  20     D-0  Eth0/0/24  100
192.168.0.107   2cea-7fed-051c  18     D-0  Eth0/0/24  100
192.168.0.99    e0d5-5efa-1b79  20     D-0  Eth0/0/24  100
192.168.0.189   8ca6-dfc4-490d  20     D-0  Eth0/0/24  100
192.168.0.11    0017-6112-b286  20     D-0  Eth0/0/24  100
192.168.0.114   f4b5-2018-50e7  20     D-0  Eth0/0/24  100
10.0.0.2        80fb-06c9-6db9         I -  Vlanif200
10.0.0.1        0c82-6858-7222  20     D-0  GE0/0/3  200
192.168.201.1   80fb-06c9-6db9         I -  Vlanif201
------------------------------------------------------------------------------
Total:173       Dynamic:151     Static:0    Interface:22   
<3700>    
'''

f = open('display-arp.template')
template = TextFSM(f)
outTab = template.ParseText(tabStr)
print (outTab)

[['192.168.11.1', '80fb-06c9-6db9', 'I - ', 'Vlanif11', ''], ['192.168.11.199', 'b42e-9968-4090', 'D-0', 'Eth0/0/1', '  11'], ['192.168.11.30', '4419-b641-7795', 'D-0', 'Eth0/0/1', '  11'], ['192.168.11.189', '0800-2740-3822', 'D-0', 'Eth0/0/1', '  11'], ['192.168.12.1', '80fb-06c9-6db9', 'I - ', 'Vlanif12', ''], ['192.168.13.1', '80fb-06c9-6db9', 'I - ', 'Vlanif13', ''], ['192.168.14.1', '80fb-06c9-6db9', 'I - ', 'Vlanif14', ''], ['192.168.14.13', '8269-c0e2-cbe1', 'D-0', 'Eth0/0/11', '  14'], ['192.168.15.1', '80fb-06c9-6db9', 'I - ', 'Vlanif15', ''], ['192.168.15.254', '8ca6-df43-6b43', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.33', '7427-eae8-fef9', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.177', '4ced-fb6c-ef9f', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.200', '408d-5c8f-2aca', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.88', 'b42e-99e6-3c49', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.63', '4ced-fb41-fba5', 'D-0', 'Eth0/0/5', '  15'], ['192.168.15.231', 'b42e-999d-1f99', 'D-0', 'Eth0/0/5

In [26]:
len(outTab)

169

In [27]:
def hwArpRows(tabStr,tpl="display-arp.template"):
    f = open(tpl)
    template = TextFSM(f)
    outTab = template.ParseText(tabStr)
    return  outTab


In [28]:
print(tabStr)

display arp
IP ADDRESS      MAC ADDRESS  EXPIRE(M) TYPE INTERFACE      VPN-INSTANCE         VLAN 
------------------------------------------------------------------------------
192.168.11.1    80fb-06c9-6db9         I -  Vlanif11
192.168.11.199  b42e-9968-4090  9      D-0  Eth0/0/1  11
192.168.11.30   4419-b641-7795  9      D-0  Eth0/0/1  11
192.168.11.189  0800-2740-3822  9      D-0  Eth0/0/1  11
192.168.12.1    80fb-06c9-6db9         I -  Vlanif12
192.168.13.1    80fb-06c9-6db9         I -  Vlanif13
192.168.14.1    80fb-06c9-6db9         I -  Vlanif14
192.168.14.13   8269-c0e2-cbe1  4      D-0  Eth0/0/11  14
192.168.15.1    80fb-06c9-6db9         I -  Vlanif15
192.168.15.254  8ca6-df43-6b43  9      D-0  Eth0/0/5  15
192.168.15.33   7427-eae8-fef9  9      D-0  Eth0/0/5  15
192.168.15.177  4ced-fb6c-ef9f  9      D-0  Eth0/0/5  15
192.168.15.200  408d-5c8f-2aca  9      D-0  Eth0/0/5  15
192.168.15.88   b42e-99e6-3c49  17     D-0  Eth0/0/5  15
192.168.15.63   4ced-fb41-fba5  17     D-0  

In [29]:
def hwMacTab(tn):
    """ 返回交换 arp表"""
    tabLs = []
    tn.write(b" display mac-address\n")
    while 1 :
        outLs = tn.expect([b"\r\n<.+?>",b"---- More ----"],timeout=5) 
        if outLs[0] == 1:
            #还有下一个页
            tn.write(b"   ")
            tabLs.append(outLs[2].decode('ascii'))
            #print(outLs[2])
        if outLs[0] <= 0:
            # 完成读取
            #print(outLs[2])
            tabLs.append(outLs[2].decode('ascii'))
            break
    return tabLs
def hwMacRows(tabStr,tpl="display-mac.template"):
    f = open(tpl)
    template = TextFSM(f)
    outTab = template.ParseText(tabStr)
    return  outTab
    
tabLs = hwMacTab(tn)
tabStr = list2str(tabLs)                          


In [30]:
print(tabStr)

                           display mac-address
MAC address table of slot 0:
-------------------------------------------------------------------------------
MAC Address    VLAN/       PEVLAN CEVLAN Port            Type      LSP/LSR-ID  
               VSI/SI                                              MAC-Tunnel  
-------------------------------------------------------------------------------
00e0-4c37-ace4 34          -      -      Eth0/0/11       dynamic   -           
e0d5-5ef7-0e13 20          -      -      Eth0/0/9        dynamic   -           
00e0-4cff-2c14 34          -      -      Eth0/0/11       dynamic   -           
7427-eae8-fef9 15          -      -      Eth0/0/5        dynamic   -           
acb9-2f24-2384 88          -      -      Eth0/0/1        dynamic   -           
e0d5-5e56-7edb 17          -      -      Eth0/0/7        dynamic   -           
5cde-3414-a887 26          -      -      Eth0/0/13       dynamic   -           
f4b5-203b-2404 26          -      -      Eth

In [31]:
outTab = hwMacRows(tabStr)
outTab

[['00e0-4c37-ace4', '34', 'Eth0/0/11'],
 ['e0d5-5ef7-0e13', '20', 'Eth0/0/9'],
 ['00e0-4cff-2c14', '34', 'Eth0/0/11'],
 ['7427-eae8-fef9', '15', 'Eth0/0/5'],
 ['acb9-2f24-2384', '88', 'Eth0/0/1'],
 ['e0d5-5e56-7edb', '17', 'Eth0/0/7'],
 ['5cde-3414-a887', '26', 'Eth0/0/13'],
 ['f4b5-203b-2404', '26', 'Eth0/0/16'],
 ['70b5-e874-5844', '34', 'Eth0/0/11'],
 ['b42e-9904-6dda', '34', 'Eth0/0/11'],
 ['f4b5-2024-f44c', '34', 'Eth0/0/11'],
 ['f000-006c-8155', '34', 'Eth0/0/11'],
 ['f000-006c-8167', '34', 'Eth0/0/11'],
 ['b42e-99e6-3c49', '15', 'Eth0/0/5'],
 ['e062-9087-a15a', '34', 'Eth0/0/11'],
 ['4419-b649-83af', '34', 'Eth0/0/11'],
 ['00e0-4f03-2e24', '34', 'Eth0/0/11'],
 ['6045-cb7f-c902', '34', 'Eth0/0/11'],
 ['e247-1f2c-0f6d', '34', 'Eth0/0/11'],
 ['b42e-999d-1f99', '15', 'Eth0/0/5'],
 ['8469-9386-4585', '34', 'Eth0/0/11'],
 ['2cea-7feb-b118', '100', 'Eth0/0/20'],
 ['e062-9082-2258', '34', 'Eth0/0/11'],
 ['408d-5c1d-9e3e', '20', 'Eth0/0/9'],
 ['4cbd-8f90-71a7', '88', 'Eth0/0/11'],
 ['0c9

In [32]:
print(tabStr)

                           display mac-address
MAC address table of slot 0:
-------------------------------------------------------------------------------
MAC Address    VLAN/       PEVLAN CEVLAN Port            Type      LSP/LSR-ID  
               VSI/SI                                              MAC-Tunnel  
-------------------------------------------------------------------------------
00e0-4c37-ace4 34          -      -      Eth0/0/11       dynamic   -           
e0d5-5ef7-0e13 20          -      -      Eth0/0/9        dynamic   -           
00e0-4cff-2c14 34          -      -      Eth0/0/11       dynamic   -           
7427-eae8-fef9 15          -      -      Eth0/0/5        dynamic   -           
acb9-2f24-2384 88          -      -      Eth0/0/1        dynamic   -           
e0d5-5e56-7edb 17          -      -      Eth0/0/7        dynamic   -           
5cde-3414-a887 26          -      -      Eth0/0/13       dynamic   -           
f4b5-203b-2404 26          -      -      Eth

In [33]:


output = '''
 display mac-address
MAC address table of slot 0:
-------------------------------------------------------------------------------
MAC Address    VLAN/       PEVLAN CEVLAN Port            Type      LSP/LSR-ID  
               VSI/SI                                              MAC-Tunnel  
-------------------------------------------------------------------------------
00e0-4c37-ace4 34          -      -      Eth0/0/11       dynamic   -           
00e0-4cff-2c14 34          -      -      Eth0/0/11       dynamic   -           
c85b-7669-9e16 20          -      -      Eth0/0/9        dynamic   -           
7427-eae8-fef9 15          -      -      Eth0/0/5        dynamic   -           
acb9-2f24-2384 88          -      -      Eth0/0/1        dynamic   -           
e0d5-5e56-7edb 17          -      -      Eth0/0/7        dynamic   -           
5cde-3414-a887 26          -      -      Eth0/0/13       dynamic   -           
f4b5-203b-2404 26          -      -      Eth0/0/16       dynamic   -           
70b5-e874-5844 34          -      -      Eth0/0/11       dynamic   -           
'''

f = open('display-mac.template')
template = TextFSM(f)
outTab = template.ParseText(tabStr)
print (outTab)

[['00e0-4c37-ace4', '34', 'Eth0/0/11'], ['e0d5-5ef7-0e13', '20', 'Eth0/0/9'], ['00e0-4cff-2c14', '34', 'Eth0/0/11'], ['7427-eae8-fef9', '15', 'Eth0/0/5'], ['acb9-2f24-2384', '88', 'Eth0/0/1'], ['e0d5-5e56-7edb', '17', 'Eth0/0/7'], ['5cde-3414-a887', '26', 'Eth0/0/13'], ['f4b5-203b-2404', '26', 'Eth0/0/16'], ['70b5-e874-5844', '34', 'Eth0/0/11'], ['b42e-9904-6dda', '34', 'Eth0/0/11'], ['f4b5-2024-f44c', '34', 'Eth0/0/11'], ['f000-006c-8155', '34', 'Eth0/0/11'], ['f000-006c-8167', '34', 'Eth0/0/11'], ['b42e-99e6-3c49', '15', 'Eth0/0/5'], ['e062-9087-a15a', '34', 'Eth0/0/11'], ['4419-b649-83af', '34', 'Eth0/0/11'], ['00e0-4f03-2e24', '34', 'Eth0/0/11'], ['6045-cb7f-c902', '34', 'Eth0/0/11'], ['e247-1f2c-0f6d', '34', 'Eth0/0/11'], ['b42e-999d-1f99', '15', 'Eth0/0/5'], ['8469-9386-4585', '34', 'Eth0/0/11'], ['2cea-7feb-b118', '100', 'Eth0/0/20'], ['e062-9082-2258', '34', 'Eth0/0/11'], ['408d-5c1d-9e3e', '20', 'Eth0/0/9'], ['4cbd-8f90-71a7', '88', 'Eth0/0/11'], ['0c9d-92cc-8c94', '26', 'Eth0

In [18]:
len(outTab)

235

In [317]:
out = tn.read_very_eager().decode('ascii')#读取缓冲区内的数据不等待
out

''

### 关闭连接


In [69]:

tn.close()

In [44]:
tmp = json.dumps(outTab, sort_keys=True, indent=4, separators=(',', ': '))
print(tmp)

[
    [
        "00e0-4c37-ace4",
        "34",
        "Eth0/0/11"
    ],
    [
        "e0d5-5ef7-0e13",
        "20",
        "Eth0/0/9"
    ],
    [
        "00e0-4cff-2c14",
        "34",
        "Eth0/0/11"
    ],
    [
        "7427-eae8-fef9",
        "15",
        "Eth0/0/5"
    ],
    [
        "acb9-2f24-2384",
        "88",
        "Eth0/0/1"
    ],
    [
        "e0d5-5e56-7edb",
        "17",
        "Eth0/0/7"
    ],
    [
        "5cde-3414-a887",
        "26",
        "Eth0/0/13"
    ],
    [
        "f4b5-203b-2404",
        "26",
        "Eth0/0/16"
    ],
    [
        "70b5-e874-5844",
        "34",
        "Eth0/0/11"
    ],
    [
        "b42e-9904-6dda",
        "34",
        "Eth0/0/11"
    ],
    [
        "f4b5-2024-f44c",
        "34",
        "Eth0/0/11"
    ],
    [
        "f000-006c-8155",
        "34",
        "Eth0/0/11"
    ],
    [
        "f000-006c-8167",
        "34",
        "Eth0/0/11"
    ],
    [
        "b42e-99e6-3c49",
        "15",
       

In [43]:
int('100')

100

In [46]:
for row in outTab:


    keys = ['mac', 'vlan', 'port']

    dictionary = dict(zip(keys, row))
    print(dictionary)

{'mac': '00e0-4c37-ace4', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'e0d5-5ef7-0e13', 'vlan': '20', 'port': 'Eth0/0/9'}
{'mac': '00e0-4cff-2c14', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '7427-eae8-fef9', 'vlan': '15', 'port': 'Eth0/0/5'}
{'mac': 'acb9-2f24-2384', 'vlan': '88', 'port': 'Eth0/0/1'}
{'mac': 'e0d5-5e56-7edb', 'vlan': '17', 'port': 'Eth0/0/7'}
{'mac': '5cde-3414-a887', 'vlan': '26', 'port': 'Eth0/0/13'}
{'mac': 'f4b5-203b-2404', 'vlan': '26', 'port': 'Eth0/0/16'}
{'mac': '70b5-e874-5844', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'b42e-9904-6dda', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f4b5-2024-f44c', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f000-006c-8155', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'f000-006c-8167', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': 'b42e-99e6-3c49', 'vlan': '15', 'port': 'Eth0/0/5'}
{'mac': 'e062-9087-a15a', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '4419-b649-83af', 'vlan': '34', 'port': 'Eth0/0/11'}
{'mac': '00e0-4f03-2e24', 'vl